### Bronze Layer

workspace.`ecommerce_analysis` : tables

In [0]:
from pyspark.sql import functions as f

source_path = "/Volumes/workspace/ecommerce/ecommerce_data/2019-Nov.csv"

# 1. Read Raw CSV
raw_df = (spark.read
          .option("header", True)
          .option("inferSchema", True)
          .csv(source_path))

# 2. Basic Bronze Cleanup (Handling Nulls and adding source info)
bronze_df = raw_df.withColumn("ingestion_ts", f.current_timestamp()) \
                  .withColumn("source_file", f.lit(source_path)) \
                  .withColumn("category_code", f.coalesce(f.col("category_code"), f.lit("unknown")))

# 3. Save as a Registered Table (This is better for SQL Warehouse)
bronze_df.write.format("delta").mode("overwrite").option("overwriteSchema", "true").saveAsTable("bronze_events")

print("Bronze Layer Created Successfully")

Bronze Layer Created Successfully


In [0]:
%sql
CREATE SCHEMA IF NOT EXISTS workspace.ecommerce_analysis;

### Silver Layer (Clean and Standardized)

In [0]:
%sql
CREATE OR REPLACE TABLE workspace.ecommerce_analysis.silver_events AS
SELECT 
    CAST(event_time AS TIMESTAMP) AS event_timestamp,
    LOWER(event_type) AS event_type,
    product_id,
    COALESCE(element_at(split(category_code, '\\.'), -1), 'unknown') AS product_name,
    category_code,
    -- FIX: Fill NULL brands with 'unknown'
    COALESCE(brand, 'unknown') AS brand, 
    CAST(price AS DECIMAL(10,2)) AS price,
    user_id,
    user_session,
    ingestion_ts
FROM bronze_events
WHERE user_id IS NOT NULL 
  AND price >= 0;

num_affected_rows,num_inserted_rows


### Verification

In [0]:
%sql
SELECT * FROM workspace.ecommerce_analysis.silver_events LIMIT 5;

event_timestamp,event_type,product_id,product_name,category_code,brand,price,user_id,user_session,ingestion_ts
2019-11-15T03:55:29.000Z,view,1307074,notebook,computers.notebook,acer,745.68,540559685,e8330a20-d1ca-b02b-ae9b-76731d7c19cf,2026-01-18T03:21:06.989Z
2019-11-15T03:55:29.000Z,view,12715384,unknown,unknown,comforser,81.08,527911031,527fa204-87fd-4e26-9c0c-d506c2f97356,2026-01-18T03:21:06.989Z
2019-11-15T03:55:29.000Z,view,1004158,smartphone,electronics.smartphone,samsung,769.13,568519600,07bdafd4-6914-4496-863e-4852b6f54008,2026-01-18T03:21:06.989Z
2019-11-15T03:55:29.000Z,view,8800048,telephone,electronics.telephone,nokia,21.79,522118249,fb35a4dd-be40-484c-ae49-a258c0b94993,2026-01-18T03:21:06.989Z
2019-11-15T03:55:29.000Z,view,2800439,refrigerators,appliances.kitchen.refrigerators,unknown,210.17,512558673,3ec304cc-3459-4b7a-b741-dc7f31a30fde,2026-01-18T03:21:06.989Z


### Gold Layer

### The Revenue & Trends Table

In [0]:
%sql
CREATE OR REPLACE TABLE workspace.ecommerce_analysis.gold_revenue_trending AS
WITH daily AS (
  SELECT 
    DATE(event_timestamp) as event_date, 
    SUM(price) as rev -- Using 'price' from your silver table as revenue
  FROM workspace.ecommerce_analysis.silver_events 
  WHERE event_type = 'purchase'
  GROUP BY 1
)
SELECT 
  event_date, 
  ROUND(rev, 2) as daily_revenue,
  -- Your moving average logic with rounding
  ROUND(AVG(rev) OVER (ORDER BY event_date ROWS BETWEEN 6 PRECEDING AND CURRENT ROW), 2) as ma7
FROM daily;

num_affected_rows,num_inserted_rows


### Verify result

In [0]:
%sql
SELECT * FROM workspace.ecommerce_analysis.gold_revenue_trending LIMIT 5

event_date,daily_revenue,ma7
2019-11-01,6949415.03,6949415.03
2019-11-02,6389808.37,6669611.70
2019-11-03,6656920.09,6665381.16
2019-11-04,8033899.65,7007510.79
2019-11-05,7250120.41,7056032.71


### Conversion Funnel

In [0]:
%sql
CREATE OR REPLACE TABLE workspace.ecommerce_analysis.gold_funnel_metrics AS
SELECT 
    category_code,
    COUNT(CASE WHEN event_type = 'view' THEN 1 END) as views,
    COUNT(CASE WHEN event_type = 'purchase' THEN 1 END) as purchases,
    ROUND(COUNT(CASE WHEN event_type = 'purchase' THEN 1 END) * 100.0 / 
          NULLIF(COUNT(CASE WHEN event_type = 'view' THEN 1 END), 0), 2) as conversion_rate
FROM workspace.ecommerce_analysis.silver_events
GROUP BY 1
ORDER BY conversion_rate DESC; -- This is the magic line

num_affected_rows,num_inserted_rows


### Verify Result

In [0]:
%sql
SELECT * FROM workspace.ecommerce_analysis.gold_funnel_metrics
LIMIT 5;

category_code,views,purchases,conversion_rate
kids.fmcg.diapers,29296,819,2.80
electronics.smartphone,14832387,382647,2.58
electronics.audio.headphone,1644910,40834,2.48
appliances.kitchen.microwave,285186,5146,1.80
appliances.personal.scales,88564,1577,1.78


### Customer Tiers

In [0]:
%sql
CREATE OR REPLACE TABLE workspace.ecommerce_analysis.gold_customer_tiers AS
SELECT
  CASE 
    WHEN cnt >= 10 THEN 'VIP'
    WHEN cnt >= 5 THEN 'Loyal'
    ELSE 'Regular' 
  END as tier,
  COUNT(*) as customers,
  ROUND(AVG(total_spent), 2) as avg_ltv
FROM (
  SELECT 
    user_id, 
    COUNT(*) as cnt, 
    SUM(price) as total_spent
  FROM workspace.ecommerce_analysis.silver_events 
  WHERE event_type = 'purchase' 
  GROUP BY user_id
)
GROUP BY 1; -- Groups by the tier

num_affected_rows,num_inserted_rows


### Verify result

In [0]:
%sql
SELECT * FROM workspace.ecommerce_analysis.gold_customer_tiers LIMIT 5;

tier,customers,avg_ltv
Regular,409374,421.85
VIP,8715,6329.12
Loyal,23549,2010.29
